# Test RNN Policy  with Engine Failure

In [1]:
import numpy as np
import os,sys



sys.path.append('../../../RL_lib/Agents/PPO')
sys.path.append('../../../RL_lib/Utils')
sys.path.append('../../../Mars3dof_env')

%load_ext autoreload
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os
print(os.getcwd())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/briangaudet/Study/Subjects/MachineLearning/Projects/RL_Meta-Learning-master/Experiments/Mars3DOF/Sensor_noise


In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

# Optimize Policy

In [5]:
from env import Env
import env_utils as envu
from dynamics_model import Dynamics_model
from lander_model_noise import Lander_model
from ic_gen2 import Landing_icgen
import rl_utils

from arch_policy_vf import Arch

from policy import Policy
from value_function import Value_function

import policy_nets as policy_nets
import valfunc_nets as valfunc_nets

from agent import Agent


import torch.nn as nn

from flat_constraint import Flat_constraint
from glideslope_constraint import Glideslope_constraint
from reward_terminal_mdr import Reward


logger = rl_utils.Logger()
dynamics_model = Dynamics_model()
lander_model = Lander_model(apf_tau1=20,apf_tau2=100,apf_vf1=-2,apf_vf2=-1,apf_v0=70,apf_atarg=15.,
                            sensor_pos_bias_range=(-0.10,0.10), sensor_vel_bias_range=(-0.10,0.10))
lander_model.get_state_agent = lander_model.get_state_agent5

obs_dim = 5
act_dim = 3
recurrent_steps = 120

reward_object = Reward()

glideslope_constraint = Glideslope_constraint(gs_limit=-1.0)
shape_constraint = Flat_constraint()
env = Env(lander_model,dynamics_model,logger,
          reward_object=reward_object,
          glideslope_constraint=glideslope_constraint,
          shape_constraint=shape_constraint,
          tf_limit=100.0,print_every=10)

env.ic_gen = Landing_icgen(mass_uncertainty=0.05, 
                           g_uncertainty=(0.05,0.05),
                           adjust_apf_v0=True,
                          downrange = (0,2000 , -70, -10), 
                           crossrange = (-1000,1000 , -30,30),  
                           altitude = (2300,2400,-90,-70))
env.ic_gen.show()

arch = Arch()

policy = Policy(policy_nets.GRU(obs_dim, act_dim, recurrent_steps=recurrent_steps), shuffle=False,
                kl_targ=0.001,epochs=20, beta=0.1, servo_kl=True, max_grad_norm=30,
                init_func=rl_utils.xn_init)
value_function = Value_function(valfunc_nets.GRU(obs_dim, recurrent_steps=recurrent_steps), 
                                shuffle=False, batch_size=9999999, max_grad_norm=30)

agent = Agent(arch, policy, value_function, None, env, logger,
              policy_episodes=30, policy_steps=3000, gamma1=0.95, gamma2=0.995, lam=0.98, 
              recurrent_steps=recurrent_steps, monitor=env.rl_stats)

load_params=False
fname = "recurrent_policy-120step"
if load_params:
    policy.load_params(fname)
    value_function.load_params(fname)
else:    
    agent.train(30000)

3-dof dynamics model
lander model apf
queue fixed
Flat Constraint


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Lander Env
Landing_icgen:
    nominal_g                   :  -3.7114
    downrange                   :  (0, 2000, -70, -10)
    crossrange                  :  (-1000, 1000, -30, 30)
    altitude                    :  (2300, 2400, -90, -70)
    adjust_apf_v0               :  True
Policy with vectorized sample
	xn_init: layer  Linear(in_features=5, out_features=50, bias=True)
	xn_init: layer  GRUCell(50, 38)
	xn_init: layer  Linear(in_features=38, out_features=30, bias=True)
	xn_init: layer  Linear(in_features=30, out_features=3, bias=True)
Policy: recurrent steps > 1, disabling shuffle
[[-1.]
 [ 0.]
 [ 1.]]
	Test Mode:          False
	Clip Param:         0.1
	Shuffle :           False
	Max Grad Norm:      30
	Recurrent Steps:    120
	Rollout Limit:      1
Value Funtion
	xn_init: layer  Linear(in_features=5, out_features=50, bias=True)
	xn_init: layer  GRUCell(50, 15)
	xn_init: layer  Linear(in_features=15, out_features=5, bias=True)
	xn_init: layer  Linear(in_features=5, out_features=1,

KeyboardInterrupt: 

In [7]:
fname = "recurrent_policy-rkill-60step"
policy.save_params(fname)
value_function.save_params(fname)
np.save(fname + "_history",env.rl_stats.history)

# Test Policy 

In [4]:
policy.test_mode=True
  
env.test_policy_batch(agent,10000,print_every=100)

i :  100
Cumulative Stats (mean,std,max,argmax)
thrust   |9814.55 |2308.32 |2000.00 |15000.00 |    10
glideslope | 5.301 |17.088 | 0.396 |1177.487 |    56
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.296 |   0.122 |   1.050 |   1.578
norm_rf  |     0.6 |     0.3 |     0.1 |     1.6
position |    -0.2    -0.0    -0.0 |     0.5     0.4     0.0 |    -0.9    -0.9    -0.1 |     1.4     1.6    -0.0
velocity |   0.030  -0.036  -1.294 |   0.043   0.043   0.123 |  -0.054  -0.111  -1.573 |   0.120   0.057  -1.048
fuel     |286.65 | 16.80 |257.33 |324.47
glideslope | 38.41 | 30.15 | 12.22 |222.74
i :  200
Cumulative Stats (mean,std,max,argmax)
thrust   |9857.55 |2300.58 |2000.00 |15000.00 |    10
glideslope | 5.679 |17.574 | 0.332 |1177.487 |    56
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.309 |   0.134 |   1.050 |   1.749
norm_rf  |     0.5 |     0.3 |     0.1 |     2.7
position |    

i :  1400
Cumulative Stats (mean,std,max,argmax)
thrust   |9878.30 |2321.96 |2000.00 |15000.00 |   918
glideslope | 5.814 |22.669 | 0.245 |4966.127 |   919
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.314 |   0.134 |   1.010 |   1.749
norm_rf  |     0.5 |     0.4 |     0.0 |     3.9
position |    -0.2    -0.0    -0.0 |     0.5     0.4     0.0 |    -1.1    -3.9    -0.1 |     3.9     2.1    -0.0
velocity |   0.037  -0.030  -1.312 |   0.046   0.046   0.133 |  -0.069  -0.120  -1.742 |   0.142   0.071  -1.010
fuel     |285.15 | 16.60 |243.55 |342.66
glideslope | 39.71 | 36.72 | 11.89 |485.00
i :  1500
Cumulative Stats (mean,std,max,argmax)
thrust   |9875.90 |2320.89 |2000.00 |15000.00 |   918
glideslope | 5.761 |22.515 | 0.245 |4966.127 |   919
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.315 |   0.133 |   1.010 |   1.749
norm_rf  |     0.5 |     0.4 |     0.0 |     3.9
position |  

i :  2700
Cumulative Stats (mean,std,max,argmax)
thrust   |9878.29 |2318.62 |2000.00 |15000.00 |   918
glideslope | 5.678 |22.450 | 0.199 |6192.728 |  1970
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.313 |   0.132 |   1.007 |   1.749
norm_rf  |     0.6 |     0.4 |     0.0 |     6.3
position |    -0.2    -0.0    -0.0 |     0.5     0.4     0.0 |    -2.2    -3.9    -0.1 |     6.3     4.1    -0.0
velocity |   0.036  -0.027  -1.311 |   0.047   0.046   0.132 |  -0.076  -0.120  -1.744 |   0.143   0.077  -1.006
fuel     |285.48 | 16.67 |242.62 |342.66
glideslope | 38.68 | 34.22 | 11.61 |485.00
i :  2800
Cumulative Stats (mean,std,max,argmax)
thrust   |9881.08 |2317.54 |2000.00 |15000.00 |   918
glideslope | 5.669 |22.200 | 0.199 |6192.728 |  1970
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.315 |   0.132 |   1.007 |   1.749
norm_rf  |     0.6 |     0.4 |     0.0 |     6.3
position |  

i :  4000
Cumulative Stats (mean,std,max,argmax)
thrust   |9878.55 |2310.67 |2000.00 |15000.00 |   918
glideslope | 5.616 |22.419 | 0.199 |7200.914 |  3278
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.315 |   0.132 |   1.007 |   1.749
norm_rf  |     0.6 |     0.4 |     0.0 |     6.3
position |    -0.2    -0.0    -0.0 |     0.5     0.4     0.0 |    -2.2    -3.9    -0.1 |     6.3     4.1    -0.0
velocity |   0.036  -0.026  -1.313 |   0.047   0.046   0.132 |  -0.076  -0.120  -1.744 |   0.143   0.077  -1.006
fuel     |285.36 | 16.66 |242.62 |344.81
glideslope | 39.31 | 41.45 | 11.42 |1412.88
i :  4100
Cumulative Stats (mean,std,max,argmax)
thrust   |9877.55 |2310.35 |2000.00 |15000.00 |   918
glideslope | 5.619 |22.332 | 0.199 |7200.914 |  3278
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.316 |   0.132 |   1.007 |   1.749
norm_rf  |     0.6 |     0.4 |     0.0 |     6.3
position | 

i :  5300
Cumulative Stats (mean,std,max,argmax)
thrust   |9879.46 |2308.48 |2000.00 |15000.00 |   918
glideslope | 5.632 |22.518 | 0.199 |7906.775 |  4619
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.316 |   0.132 |   1.004 |   1.749
norm_rf  |     0.6 |     0.4 |     0.0 |     6.3
position |    -0.2    -0.0    -0.0 |     0.5     0.4     0.0 |    -2.2    -3.9    -0.1 |     6.3     4.1    -0.0
velocity |   0.036  -0.026  -1.314 |   0.047   0.046   0.131 |  -0.076  -0.124  -1.744 |   0.149   0.080  -1.003
fuel     |285.24 | 16.87 |236.88 |347.85
glideslope | 41.23 |116.49 | 11.42 |7906.78
i :  5400
Cumulative Stats (mean,std,max,argmax)
thrust   |9877.67 |2308.18 |2000.00 |15000.00 |   918
glideslope | 5.619 |22.416 | 0.199 |7906.775 |  4619
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.316 |   0.131 |   1.004 |   1.749
norm_rf  |     0.6 |     0.4 |     0.0 |     6.3
position | 

i :  6600
Cumulative Stats (mean,std,max,argmax)
thrust   |9876.22 |2308.82 |2000.00 |15000.00 |  6068
glideslope | 5.627 |22.016 | 0.199 |7906.775 |  4619
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.315 |   0.131 |   1.004 |   1.749
norm_rf  |     0.6 |     0.4 |     0.0 |     6.3
position |    -0.2     0.0    -0.0 |     0.5     0.4     0.0 |    -2.2    -3.9    -0.1 |     6.3     4.1    -0.0
velocity |   0.036  -0.026  -1.313 |   0.047   0.046   0.131 |  -0.076  -0.124  -1.744 |   0.149   0.080  -1.003
fuel     |285.32 | 16.93 |236.88 |366.48
glideslope | 40.94 |106.53 |  9.94 |7906.78
i :  6700
Cumulative Stats (mean,std,max,argmax)
thrust   |9878.13 |2309.78 |2000.00 |15000.00 |  6068
glideslope | 5.626 |21.978 | 0.199 |7906.775 |  4619
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.316 |   0.131 |   1.004 |   1.749
norm_rf  |     0.6 |     0.4 |     0.0 |     6.3
position | 

i :  7900
Cumulative Stats (mean,std,max,argmax)
thrust   |9879.55 |2310.28 |2000.00 |15000.00 |  6068
glideslope | 5.639 |22.000 | 0.199 |7906.775 |  4619
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.316 |   0.132 |   0.974 |   1.749
norm_rf  |     0.6 |     0.4 |     0.0 |     6.3
position |    -0.2     0.0    -0.0 |     0.6     0.4     0.0 |    -2.2    -3.9    -0.1 |     6.3     4.1    -0.0
velocity |   0.036  -0.026  -1.313 |   0.047   0.046   0.131 |  -0.076  -0.124  -1.746 |   0.155   0.080  -0.968
fuel     |285.33 | 16.95 |236.88 |366.48
glideslope | 40.94 | 99.08 |  9.94 |7906.78
i :  8000
Cumulative Stats (mean,std,max,argmax)
thrust   |9880.36 |2310.39 |2000.00 |15000.00 |  6068
glideslope | 5.633 |21.929 | 0.199 |7906.775 |  4619
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.316 |   0.131 |   0.974 |   1.749
norm_rf  |     0.6 |     0.4 |     0.0 |     6.3
position | 

i :  9200
Cumulative Stats (mean,std,max,argmax)
thrust   |9881.06 |2311.82 |2000.00 |15000.00 |  6068
glideslope | 5.640 |21.584 | 0.199 |7906.775 |  4619
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.316 |   0.132 |   0.974 |   1.749
norm_rf  |     0.6 |     0.4 |     0.0 |     6.3
position |    -0.2     0.0    -0.0 |     0.5     0.4     0.0 |    -2.2    -3.9    -0.1 |     6.3     4.7    -0.0
velocity |   0.036  -0.025  -1.313 |   0.047   0.046   0.132 |  -0.076  -0.124  -1.746 |   0.156   0.080  -0.968
fuel     |285.38 | 16.95 |236.88 |366.48
glideslope | 40.93 | 93.70 |  9.94 |7906.78
i :  9300
Cumulative Stats (mean,std,max,argmax)
thrust   |9880.21 |2311.34 |2000.00 |15000.00 |  6068
glideslope | 5.637 |21.543 | 0.199 |7906.775 |  4619
sc_margin |100.000 | 0.000 |100.000 |100.000 |     0

Final Stats (mean,std,min,max)
norm_vf  |   1.316 |   0.131 |   0.974 |   1.749
norm_rf  |     0.6 |     0.4 |     0.0 |     6.4
position | 

In [9]:
len(lander_model.trajectory_list)
traj_list = lander_model.trajectory_list[0:100]
len(traj_list)
np.save('recurrent_policy-rkill-60step_100traj',traj_list)

In [10]:
v1 = 1
v2 = 1.2
v3 = v1*1.1
v4 = v2*1.1
print(v1-v2,v3-v4)

-0.19999999999999996 -0.21999999999999997
